In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import time
from tqdm import tqdm
#from models import * #model and dataset
from train import *

if torch.cuda.is_available():  
    torch.cuda.empty_cache()
    device = "cuda:0" 
else:  
    device = "cpu" 
    
print(torch.__version__)
print(torchvision.__version__)

C:\Users\jerry\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.10.2
0.11.3


In [2]:
model = Double_UNet()
n = torch.rand(1,3,256,256)
print(model(n).shape)
print(device)

torch.Size([1, 1, 256, 256])
cuda:0


In [4]:
imgpath = 'Kvasir-SEG/images'
maskpath = 'Kvasir-SEG/masks'

imgpathv = imgpath#'Result/valid_images'
maskpathv = maskpath#'Result/valid_masks'

savepath='./Result/'

load=False

if load:
    model = torch.load('model.pt').to(device)
    '''
    trainloss = #load from txt
    validloss = #load from txt
    best_IOU = #load from txt
    cur_epoch = #load from txt
    #optimizer
    '''
else:
    model=UNet().to(device)
    trainloss=[]
    validloss=[]
    best_IOU=0
    cur_epoch=0
    batch = 2
trainset = Images(imgpath,maskpath)
train_loader=DataLoader(trainset,batch,shuffle=True)

validset = Images(imgpathv,maskpathv)
valid_loader = DataLoader(validset,batch,shuffle=True)

optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_function = DiceLoss()
#loss_function = nn.BCELoss()
epoch=1
start = time.time()

for i in range(cur_epoch,epoch):
    train_accuracy,train_loss,train_IOU,train_dice=train(model,train_loader,loss_function,optimizer,device,len(trainset),batch)
    trainloss.append(train_loss)
    #torch.save(model,'model/model.pt')
    
    valid_accuracy, valid_loss,valid_IOU, valid_dice=valid(model,valid_loader,loss_function,device,len(validset),batch)
    validloss.append(valid_loss)
    
    #save best model
    if valid_IOU > best_IOU:
        #torch.save(model,savepath+'best.pt')
        save_report('UNet','BCE',epoch,train_accuracy,train_IOU,train_dice,train_loss.item(),valid_accuracy,valid_IOU,valid_dice,valid_loss.item())
        best_IOU=valid_IOU
    print('epoch '+str(i+1)+':')
    print('train: '+str(round(train_accuracy,5))+' tr_loss:',str(round(train_loss.item(),5))
          +' tr_IOU:'+str(round(train_IOU,5))+' tr_Dice:'+str(round(train_dice,5)))
    print('valid: '+str(round(valid_accuracy,5))+' vl_loss:',str(round(valid_loss.item(),5))
         +' vl_IOU:'+str(round(valid_IOU,5))+' vl_Dice:'+str(round(valid_dice,5)))
    plt.plot(trainloss,label='train')
    plt.plot(validloss,label='valid')
    plt.legend()
    #plt.legend(["train"],label='tr')
    #plt.legend(["valid"],label='vl')
    #plt.savefig('loss.png')
    plt.close()


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [07:07<00:00,  1.17it/s]


epoch 1:
train: 0.6531 tr_loss: 0.69863 tr_IOU:0.25664 tr_Dice:0.40845
valid: 0.66294 vl_loss: 0.67142 vl_IOU:0.27021 vl_Dice:0.42545


In [2]:
mp='Result/model'
save_path = './Result/'
models = os.listdir(mp)
for model in models:
    npt = model[:-3]
    if not os.path.exists(save_path+npt):
        os.mkdir(save_path+npt)
    save_mask(os.path.join(mp,model),'Result/valid_images',save_path+npt)
    encode = os.listdir(save_path+npt)
    for img in encode:
        fn = os.path.join(save_path+npt,img)#filename
        nfn = fn[:-4]+'_'+npt+'.jpg'
        os.rename(fn,nfn)

100%|██████████| 100/100 [00:14<00:00,  6.80it/s]


In [ ]:
encode = os.listdir('Result2/valid_masks/')
for img in encode:
    fn = os.path.join('Result2/valid_masks/', img)#filename
    nfn = fn[:-4]+'_'+'a.jpg'
    os.rename(fn,nfn)